## Fa2020 CSE258 HW2
Name: Tzu-Han Cheng

PID: A53238806

email: tzcheng@ucsd.edu

In [1]:
import numpy as np        # linear algebra
import urllib.request # load data from the web
import scipy.optimize # optimization routines
import random         # random number generation
import ast
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import metrics # confusion matrix, MSE etc.
from collections import defaultdict

In [2]:
def parseData(fname):
  for l in urllib.request.urlopen(fname):
    yield eval(l)

In [3]:
def getlen(datum):
  textlen = len(datum)
  return textlen

In [4]:
data = list(parseData("https://cseweb.ucsd.edu/classes/fa20/cse258-a/data/beer_50000.json"))

In [5]:
# Random shuffle the whole dataset
random.shuffle(data)

In [65]:
Y = [1 if d['beer/ABV'] > 7.0 else -1 for d in data]
Y = np.array(Y, dtype = 'int')
style = [d["beer/style"] for d in data]

In [7]:
categoryCounts = defaultdict(int)
for d in data: 
    categoryCounts[d["beer/style"]] += 1
categories = [c for c in categoryCounts if categoryCounts[c] > 1000]
catID = dict(zip(list(categories),range(1,len(categories)+1))) # code the styles ordinally from 1 to 13, don

In [8]:
## One hot encoding
style_ordinal = [catID.get(x) if x in categories else 0 for x in style]
style_ordinal = np.array(style_ordinal, dtype = 'int')
style_ordinal = style_ordinal.reshape(-1,1)
enc = preprocessing.OneHotEncoder(drop='first')
enc.fit(style_ordinal)
onehotlabels = enc.transform(style_ordinal).toarray()

### Question 1

In [41]:
## Create feature array 
X = []
X = np.insert(onehotlabels,0,1,axis=1) # Adding 1 as the intercept

In [79]:
## Linear regression model
model_line = linear_model.LinearRegression()
model_line.fit(X_train,Y_train)
yPred_line = model_line.predict(X_test)
yPred_line = np.ndarray.tolist(yPred_line)
yPred_01 = [1 if x > 0 else -1 for x in yPred_line] 
yPred_01 = np.array(yPred_01, dtype = 'int')

In [76]:
print(yPred_line[:10])
print(yPred_01[:10])
print(yPred_logi[:10])

[-0.3171050184376303, 0.9322462394738242, -0.516910894995692, 1.1976587130204701, -0.32168072283280236, -0.2461450286393092, -0.6131152145128993, -0.20917848191237887, 0.6628969852205651, 0.9357211297773753]
[-1  1 -1  1 -1 -1 -1 -1  1  1]
[-1  1 -1  1 -1 -1 -1 -1  1  1]


In [80]:
## Logistic regression model
model_logi = linear_model.LogisticRegression(max_iter = 1000)
model_logi.fit(X_train,Y_train)
yPred_logi = model_logi.predict(X_test)

In [81]:
TP_ = np.logical_and(Y_test == 1, yPred_logi == 1)
FP_ = np.logical_and(Y_test == -1, yPred_logi == 1)
TN_ = np.logical_and(Y_test == -1, yPred_logi == -1)
FN_ = np.logical_and(Y_test == 1, yPred_logi == -1)
TP = sum(TP_)
FP = sum(FP_)
TN = sum(TN_)
FN = sum(FN_)

In [82]:
TP

8544

In [83]:
# Accuracy and BER
accuracy = (TP + TN) / (TP + FP + TN + FN)
print('Accuracy:',accuracy)
BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
print('BER:',BER)

Accuracy: 0.85536
BER: 0.15254579864232343


In [85]:
TP_ = np.logical_and(Y_test == 1, yPred_01 == 1)
FP_ = np.logical_and(Y_test == -1, yPred_01 == 1)
TN_ = np.logical_and(Y_test == -1, yPred_01 == -1)
FN_ = np.logical_and(Y_test == 1, yPred_01 == -1)
TP = sum(TP_)
FP = sum(FP_)
TN = sum(TN_)
FN = sum(FN_)

In [86]:
sum(yPred_01 == yPred_logi)

24108

In [87]:
# Accuracy and BER
accuracy = (TP + TN) / (TP + FP + TN + FN)
print('Accuracy:',accuracy)
BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
print('BER:',BER)

Accuracy: 0.84872
BER: 0.16173218141114942


In [77]:
## split train test set
N = len(Y)
X_train = np.array(X[:N//2], dtype='float') # force N//2 to be integer
X_test = np.array(X[N//2:], dtype='float')
Y_train = np.array(Y[:N//2], dtype='float')
Y_test = np.array(Y[N//2:], dtype='float')

In [ ]:
model = linear_model.LogisticRegression(C=10,class_weight='balanced')
model.fit(X_train,Y_train)
pred = model.predict(X_test)

In [ ]:
TP_ = np.logical_and(Y_test == 1, pred == 1)
FP_ = np.logical_and(Y_test == 0, pred == 1)
TN_ = np.logical_and(Y_test == 0, pred == 0)
FN_ = np.logical_and(Y_test == 1, pred == 0)
TP = sum(TP_)
FP = sum(FP_)
TN = sum(TN_)
FN = sum(FN_)

In [ ]:
# Accuracy and BER
accuracy = (TP + TN) / (TP + FP + TN + FN)
print('Accuracy:',accuracy)
BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
print('BER:',BER)

### Question 2

In [51]:
x_extra1 = [d["review/aroma"] for d in data]

In [52]:
length = [getlen(d['review/text']) for d in data]
length = np.array(length)/max(length) # normalize the length feature 
x_extra2 = [l for l in length]

In [53]:
Xr,Xc = X.shape
X = np.insert(X,Xc,[x_extra1],axis=1) 
Xr,Xc = X.shape
X = np.insert(X,Xc,[x_extra2],axis=1) 

In [ ]:
# Get training and testing subsets
N = len(Y)
X_train = np.array(X[:N//2], dtype='float') # force N//2 to be integer
X_test = np.array(X[N//2:], dtype='float')

In [ ]:
model = linear_model.LogisticRegression(C=10,class_weight='balanced',max_iter = 1000)
model.fit(X_train,Y_train)
pred = model.predict(X_test)

In [ ]:
TP = sum(np.logical_and(Y_test == 1, pred == 1))
FP = sum(np.logical_and(Y_test == 0, pred == 1))
TN = sum(np.logical_and(Y_test == 0, pred == 0))
FN = sum(np.logical_and(Y_test == 1, pred == 0))

In [ ]:
# BER
BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
print('BER:',BER)

### Question 3

In [ ]:
Nv = len(Y_test)
X_testv = np.array(X_test[:Nv//2], dtype='float') 
Y_testv = np.array(Y_test[:Nv//2], dtype='float')
X_valid = np.array(X_test[Nv//2:], dtype='float') 
Y_valid = np.array(Y_test[Nv//2:], dtype='float')

In [ ]:
C = [10**-6,10**-5,10**-4,10**-3]

In [ ]:
for i in range(0,len(C)):
    model = linear_model.LogisticRegression(C=C[i],class_weight='balanced')
    model.fit(X_train,Y_train)
    pred_train = model.predict(X_train)
    pred_valid = model.predict(X_valid)
    pred_test = model.predict(X_testv)
    TP = sum(np.logical_and(Y_train == 1, pred_train == 1))
    FP = sum(np.logical_and(Y_train == 0, pred_train == 1))
    TN = sum(np.logical_and(Y_train == 0, pred_train == 0))
    FN = sum(np.logical_and(Y_train == 1, pred_train == 0))
    BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
    print('C:',C[i],'Train BER',BER)
    TP = sum(np.logical_and(Y_valid == 1, pred_valid == 1))
    FP = sum(np.logical_and(Y_valid == 0, pred_valid == 1))
    TN = sum(np.logical_and(Y_valid == 0, pred_valid == 0))
    FN = sum(np.logical_and(Y_valid == 1, pred_valid == 0))
    BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
    print('C:',C[i],'Validation BER:',BER)
    TP = sum(np.logical_and(Y_testv == 1, pred_test == 1))
    FP = sum(np.logical_and(Y_testv == 0, pred_test == 1))
    TN = sum(np.logical_and(Y_testv == 0, pred_test == 0))
    FN = sum(np.logical_and(Y_testv == 1, pred_test == 0))
    BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
    print('C:',C[i],'Test BER:',BER)

I would select C = 0.001 to train the classifier since it provides the lowest BER for the validation set. It also provides the lowest BER for the training set. Taken together, this model could generalized well to datasets outside of the training data, meanwhile it does not underfit the training data.

### Question 5

In [ ]:
## Calculate precision and recall on the test set for C = 0.001 model
print(model)
precision = TP/(TP+FP)
recall = TP/(TP+FN)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

In [ ]:
disp = plot_precision_recall_curve(model, X_testv, Y_testv)

### Question 6

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import urllib.request

In [ ]:
edges = set()
nodes = set()
for edge in open("/Users/t.z.cheng/Google_Drive/Coursework/CSE258/HW/hw2/egonet.txt", 'r'):
    x,y = edge.split()
    x,y = int(x),int(y)
    edges.add((x,y))
    edges.add((y,x))
    nodes.add(x)
    nodes.add(y)

In [ ]:
G = nx.Graph()
for e in edges:
  G.add_edge(e[0],e[1])
nx.draw(G)
plt.show()
plt.clf()

In [ ]:
cc = nx.connected_components(G)
print('Number of connected components:',len(list(cc)))
largest_cc = max(nx.connected_components(G), key=len)
print('Number of nodes in the largest connected components:',len(largest_cc))

### Question 7

In [ ]:
cc = nx.connected_components(G)
cc = list(cc)
lc = cc[0]

In [ ]:
slc = sorted(lc)
half1 = slc[:len(lc)//2]
half2 = slc[len(lc)//2:]

In [ ]:
## Keep only the largest connected component (does not change the results)
G.remove_nodes_from([n for n in G if n not in set(lc)])
## Calculate the normalized cut cost
ncost = 1/2*nx.algorithms.cuts.normalized_cut_size(G, half1, half2) 
print('Normalized cut cost:', ncost)

### Question 8

In [ ]:
half1_origin = half1.copy()
half2_origin = half2.copy()

In [ ]:
ncost = []
for h1 in range(0,len(half1_origin)):
    for h2 in range(0,len(half2_origin)):
        half1[h1] = half2_origin[h2]
        half2[h2] = half1_origin[h1]
        tmp_ncost = 1/2*nx.algorithms.cuts.normalized_cut_size(G, half1, half2)
        ncost.append(tmp_ncost)
        half1 = half1_origin.copy()
        half2 = half2_origin.copy()

In [ ]:
## Find the min normalized cut cost
min(ncost)
ind = ncost.index(min(ncost))
h1 = ind // len(half1_origin)
h2 = ind % len(half1_origin)
half1[h1] = half2_origin[h2]
half2[h2] = half1_origin[h1]
print('The elements of the split:', half1, half2)
print('Normalized cut cost:', min(ncost))